In [ ]:
%load_ext autoreload
%autoreload 2


import xarray as xr
import xwrf
import numpy as np
import xesmf as xe

from postprocess_wrf import postprocess_wrf

# Load in wrf

In [ ]:
wrf_location = "/home/eric/data/forecast_output/wrfout_d02_2022-08-20_00:00:00"


In [ ]:
! ls /home/eric/data/forecast_output/

In [ ]:
wrf_location = "/home/eric/data/forecast_output/wrfout_d02_2022-08-01_00:00:00_processed.nc"
ds_wrf = xr.open_mfdataset(wrf_location, engine='netcdf4')

ds_wrf = ds_wrf.xwrf.postprocess()
ds_wrf = ds_wrf.xwrf.destagger()

In [ ]:
ds_wrf = ds_wrf.rename({"XLONG": "lon", "XLAT": "lat"})

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(
            ((ds_wrf.lat.min().values+0.6)/5).round(1)*5, 
            ((ds_wrf.lat.max().values-0.6)/5).round(1)*5, 
            0.25
        ), {"units": "degrees_north"}),
        "lon": (["lon"], np.arange(
            ((ds_wrf.lon.min().values+0.9)/5).round(1)*5, 
            ((ds_wrf.lon.max().values-0.6)/5).round(1)*5, 
            0.25
        ), {"units": "degrees_east"}),
    }
)
ds_out

In [ ]:
regridder = xe.Regridder(ds_wrf, ds_out, "bilinear")
regridder  # print basic regridder information.

In [ ]:

ds_wrf_out = regridder(ds_wrf, keep_attrs=True)
ds_wrf_out

# Standardizing height levels

In [ ]:
wrf_location = "/home/eric/data/forecast_output/wrfout_d02_2022-08-01_00:00:00"
ds_wrf = xr.open_mfdataset(wrf_location)

In [ ]:
ds_wrf

In [ ]:
ds_wrf_out = postprocess_wrf(wrf_location)

In [ ]:
ds_wrf_out

# Doing the above in a single method:

In [ ]:
ds_wrf_out = postprocess_wrf(wrf_location)

In [ ]:
ds_wrf_out['V'][13, 0, :, :].plot()

In [ ]:
(ds_wrf_out['T2']-273.15).clip(0)[17, :, :].plot()

In [ ]:
ds_wrf_out = ds_wrf_out.drop(['XTIME', 'x_stag', 'y_stag', 'z_stag'])

In [ ]:
ds_wrf_out[['U', 'V', 'T2', 'U10', 'V10']]

In [ ]:
ds_wrf_out[['U', 'V', 'T2', 'U10', 'V10']].to_netcdf("/home/eric/data/forecast_output_processed/wrfout.nc")